In [1]:
from binance import Client
import websockets
import json
import pandas as pd
import asyncio
import time
import threading
from colorama import Fore, Back, Style

In [2]:
api_key = 'HebcaWIceNKowmEbtQBkAddeW8rEjMUhD5Mmi3aV8WXlcu1rDITVhOx0NvH8tLBk'
api_secret = 'bQAXOTWFDpdGAtcIQozQG4vPDKzmflaAMg9U48ncvC0VngyqBhuyiJXI6zElbB57'

In [3]:
client = Client(api_key, api_secret)

In [4]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','c','v']]
    df.columns = ['Symbol','Time','Price','Volume']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [5]:
def get_sma(l, p):
    return sum(l[-p:])/p

In [6]:
coins = ['dogeusdt','btcusdt','ethusdt','linkusdt','uniusdt','algousdt','maticusdt','compusdt','lunausdt','dodousdt','ankrusdt','milusdt','gtcusdt','ampusdt','aaveusdt','adausdt','cakeusdt','bnbusdt','crvusdt','xlmusdt','sushiusdt','grtusdt']
price_percentage_change = 0.05 / 100
volume_percentage_change = 0.01 / 100
interval = 30 # interval in seconds
ma_points = 10 # number of moving average points to use

async def get_data(pair):
    first_time = True
    vol_sig = False
    price_sig = False
    
    current = {}
    previous = {}
    stream = websockets.connect(f'wss://stream.binance.com:9443/stream?streams={pair}@miniTicker')
    while(True):
        async with stream as receiver:   
            data = await receiver.recv()
            data = json.loads(data)['data']
            if first_time == True:
                df = createframe(data)
                first_time = False
                account = {
                    "Symbol": df.iloc[0]['Symbol'],
                    "Balance": 1000,
                    "Holdings": 0
                }
            else:
                df = df.append(createframe(data), ignore_index=True)
            
            if len(df.index) > ma_points:
                ma = get_sma(df["Price"], ma_points)
                curr_price = float(df.iloc[-1]["Price"])
                prev_price = float(df.iloc[-2]["Price"])
                curr_vol = float(df.iloc[-1]["Volume"])
                prev_vol = float(df.iloc[-2]["Volume"])
                
                time = pd.to_datetime(df.iloc[-1]["Time"], unit="ms")
                if curr_price > prev_price + (prev_price * price_percentage_change):
                    price_sig = True
#                     print(Fore.GREEN)
#                     print(f"{time}: {df.iloc[0]['Symbol']} rose from {prev_price} to {curr_price}")
#                     print(Style.RESET_ALL)
                if curr_vol > prev_vol + (prev_vol * volume_percentage_change):
                    vol_sig = True
#                     print(f"{time}: volume of {df.iloc[0]['Symbol']} rose by at least {volume_percentage_change * 100}%")

                if curr_price > ma and price_sig and vol_sig:
                    print(Fore.GREEN)
                    stocks = 100/curr_price
                    account["Holdings"] = account["Holdings"] + stocks
                    account["Balance"] = account["Balance"] - 100
                    print(f"Bought {stocks} units of {df.iloc[0]['Symbol']} at {curr_price}")
                    print(account)
                    print("Total bag: ", account["Holdings"] * curr_price + account["Balance"])
                    print(Style.RESET_ALL)
                    price_sig = False
                    vol_sig = False
                    
                if curr_price < ma and account["Holdings"] != 0:
                    print(Fore.RED)
                    print(f"Sold account['Holdings'] units of {df.iloc[0]['Symbol']} at {curr_price}")
                    account["Balance"] = account["Balance"] + account["Holdings"] * curr_price
                    account["Holdings"] = 0
                    print(account)
                    print("Total monies: ", account["Balance"])
                    print(Style.RESET_ALL)
                    price_sig = False
                    vol_sig = False
                      
            await asyncio.sleep(interval)
        
def wrapper_func(pair):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(get_data(pair))
    loop.close()

In [ ]:
threads = []
for coin in coins:
    t = threading.Thread(target=wrapper_func, args=(coin, ))
    threads.append(t)
    t.start()
    
for thread in threads:
    thread.join()

# 